In [1]:
import numpy as np
import pandas as pd
import re

## Import Files

In [2]:
nb_merge = pd.read_csv("notebook_dataset.csv", encoding = 'unicode_escape')

In [ ]:
nb = pd.read_csv("laptops.csv", encoding = 'unicode_escape')

In [107]:
import os
benchmark_files = os.path.join(os.getcwd(), "benchmarks")
def get_benchmark_df(file_name): return  pd.read_csv(os.path.join(benchmark_files, file_name), encoding = 'unicode_escape')
cpu = get_benchmark_df("CPU_UserBenchmarks.csv")
gpu = get_benchmark_df("GPU_UserBenchmarks.csv")

## Identify Performance

In [108]:
nb.head()

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


### CPU Benchmarks

In [126]:
cpu[cpu['Model'].str.contains('7200')].head()


,Type,Part Number,Brand,Model,Rank,Benchmark,Samples,URL
589,CPU,NaN,Intel,Core i5 7200U,590,57.6,212369,https://cpu.userbenchmark.com/SpeedTest/153577...
1028,CPU,NaN,AMD,A8 7200P APU (2014 M.Ka),1029,39.4,130,https://cpu.userbenchmark.com/SpeedTest/64755/...
1032,CPU,NaN,Intel,Core2 Duo E7200,1033,39.2,5843,https://cpu.userbenchmark.com/SpeedTest/1876/I...
1181,CPU,NaN,Intel,Core2 T7200,1182,33.6,2298,https://cpu.userbenchmark.com/SpeedTest/248/In...


In [127]:
# Get CPU columns into the same format 
nb['Cpu'] = nb['Cpu'].astype('str')
cpu["Model"] = cpu["Model"].astype('str')
nb["Cpu"] = nb["Cpu"].apply(lambda x: re.sub("AMD|Intel|([\d\.]+GHz)|(Dual Core)|(Quad Core)|", "", x.strip()))
cpu["Model"] = cpu["Model"].apply(lambda x: re.sub("-", " ", x.strip()))

In [114]:
cpu_benchmarks = cpu[["Model","Benchmark"]]

In [128]:
nb_merge = nb.merge(cpu_benchmarks, left_on="Cpu",right_on="Model")#, how='left')
# Ignore all items which don't have clearly specified CPU

In [129]:
nb_merge

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Approx Gpu,Model,Benchmark
0,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Core i5 7200U,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,UHD Graphics 605,Core i5 7200U,57.6
1,11,HP,250 G6,Notebook,15.6,1366x768,Core i5 7200U,4GB,500GB HDD,Intel HD Graphics 620,No OS,1.86kg,393.90,UHD Graphics 605,Core i5 7200U,57.6
2,39,HP,250 G6,Notebook,15.6,1366x768,Core i5 7200U,4GB,1TB HDD,Intel HD Graphics 620,Windows 10,1.86kg,488.69,UHD Graphics 605,Core i5 7200U,57.6
3,43,Asus,X542UQ-GO005 (i5-7200U/8GB/1TB/GeForce,Notebook,15.6,1366x768,Core i5 7200U,8GB,1TB HDD,Nvidia GeForce 940MX,Linux,2.3kg,522.99,GeForce 9400M,Core i5 7200U,57.6
4,49,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Core i5 7200U,4GB,256GB SSD,AMD Radeon R5 M430,Windows 10,2.3kg,639.00,Radeon 540,Core i5 7200U,57.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,954,HP,Omen 17-W006na,Gaming,17.3,IPS Panel Full HD 1920x1080,Core i5 6300HQ,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 965M,Windows 10,2.75kg,1399.00,GeForce GTX 645,Core i5 6300HQ,64.1
1056,1091,Lenovo,Ideapad 700-15ISK,Notebook,15.6,IPS Panel Full HD 1920x1080,Core i5 6300HQ,4GB,1TB HDD,Nvidia GeForce GTX 950M,Windows 10,2.3kg,977.00,GeForce GTX 580M,Core i5 6300HQ,64.1
1057,1144,HP,Omen -,Gaming,17.3,IPS Panel Full HD 1920x1080,Core i5 6300HQ,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,3.35kg,1129.00,GeForce GTX 560M,Core i5 6300HQ,64.1
1058,1172,Dell,XPS 15,Notebook,15.6,IPS Panel Touchscreen / 4K Ultra HD 3840x2160,Core i5 6300HQ,8GB,256GB SSD,Nvidia GeForce 960M,Windows 10,2.04kg,2250.68,GeForce 610M,Core i5 6300HQ,64.1


### GPU Benchmarks

In [130]:
#gpu.sample(5)
nb["Gpu"].head()

0    Intel Iris Plus Graphics 640
1          Intel HD Graphics 6000
2           Intel HD Graphics 620
3              AMD Radeon Pro 455
4    Intel Iris Plus Graphics 650
Name: Gpu, dtype: object

In [131]:
from difflib import get_close_matches 
#gpu[gpu['Model'].str.contains('620')].head()
matches = nb["Gpu"].apply(lambda x: get_close_matches(x, gpu["Model"]))

In [132]:
print(matches)

0                                          [Iris Plus G4]
1       [UHD Graphics 600, HD Graphics P4600, UHD Grap...
2       [UHD Graphics 605, UHD Graphics 600, HD Graphi...
3       [Radeon Pro 5500M, Radeon Pro 5300M, Radeon HD...
4                                                      []
                              ...                        
1298    [UHD Graphics 750, UHD Graphics 770, UHD Graph...
1299    [UHD Graphics 750, UHD Graphics 770, UHD Graph...
1300    [UHD Graphics 770, UHD Graphics 750, UHD Graph...
1301        [Radeon  530, Radeon HD 8330, Radeon HD 3300]
1302    [UHD Graphics 770, UHD Graphics 750, UHD Graph...
Name: Gpu, Length: 1303, dtype: object


In [133]:
match = matches.apply(lambda x: x[0] if len(x) > 0 else "")
nb_merge["Approx Gpu"] = match
gpu_benchmarks = gpu[["Model","Benchmark"]]
nb_merge = nb_merge.merge(gpu_benchmarks, left_on="Approx Gpu",right_on="Model")

In [134]:
nb_merge

,Unnamed: 0,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,Approx Gpu,Model_x,Benchmark_x,Model_y,Benchmark_y
0,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Core i5 7200U,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00,Iris Plus G4,Core i5 7200U,57.6,Iris Plus G4,7.26
1,133,Dell,Inspiron 5567,Notebook,15.6,1366x768,Core i5 7200U,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,638.99,Iris Plus G4,Core i5 7200U,57.6,Iris Plus G4,7.26
2,804,Lenovo,ThinkPad X1,2 in 1 Convertible,14.0,IPS Panel Touchscreen 2560x1440,Core i5 7200U,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.42kg,2330.00,Iris Plus G4,Core i5 7200U,57.6,Iris Plus G4,7.26
3,375,HP,Elitebook 1040,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Core i7 7500U,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.36kg,1750.00,Iris Plus G4,Core i7 7500U,58.9,Iris Plus G4,7.26
4,445,Lenovo,ThinkPad T470s,Ultrabook,14.0,2560x1440,Core i7 7500U,24GB,512GB SSD,Intel HD Graphics 620,Windows 10,1.32kg,2382.00,Iris Plus G4,Core i7 7500U,58.9,Iris Plus G4,7.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,365,Dell,Latitude 7480,Notebook,14.0,Full HD 1920x1080,Core i5 7300U,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.36kg,1427.00,Quadro 1000M,Core i5 7300U,56.9,Quadro 1000M,3.35
1051,1100,HP,EliteBook 840,Notebook,14.0,Full HD 1920x1080,Core i5 6200U,4GB,500GB HDD,Intel HD Graphics 520,Windows 7,1.54kg,1030.99,GeForce 210,Core i5 6200U,53.0,GeForce 210,0.77
1052,1267,Dell,XPS 13,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Core i5 7Y54,8GB,256GB SSD,Intel HD Graphics 615,Windows 10,1.24kg,1813.00,GeForce GT 630M,Core i5 7Y54,46.4,GeForce GT 630M,3.45
1053,839,Razer,Blade Pro,Gaming,17.3,4K Ultra HD / Touchscreen 3840x2160,Core i7 7820HK,32GB,512GB SSD,Nvidia GeForce GTX 1080,Windows 10,3.49kg,5499.00,Radeon HD 4650,Core i7 7820HK,73.4,Radeon HD 4650,2.89


In [136]:
# Turn weight into number
nb_merge["Weight"] = nb_merge["Weight"].apply(lambda x: float(x[:-2]))
# Convert euros to dollars is not included here because they're about on par at the moment and I'm tired

In [140]:
nb_merge["Ram"] = nb_merge["Ram"].apply(lambda x: float(x[:-2]))

In [151]:
nb_merge["Pixels"] = nb_merge["ScreenResolution"].apply(lambda x: int(re.search(r'\d+', x).group()))

In [156]:
# Get percentiles
from scipy import stats
nb_merge["Cpu percentile"] = nb_merge["Benchmark_x"].apply(lambda x: stats.percentileofscore(nb_merge["Benchmark_x"], x)/100)
nb_merge["Gpu percentile"] = nb_merge["Benchmark_y"].apply(lambda x: stats.percentileofscore(nb_merge["Benchmark_y"], x)/100)
nb_merge["Weight percentile"] = nb_merge["Weight"].apply(lambda x: stats.percentileofscore(nb_merge["Weight"], x)/100)
nb_merge["Ram percentile"] = nb_merge["Ram"].apply(lambda x: stats.percentileofscore(nb_merge["Ram"], x)/100)
nb_merge["Size percentile"] = nb_merge["Inches"].apply(lambda x: stats.percentileofscore(nb_merge["Inches"], x)/100)
nb_merge["Pixel percentile"] = nb_merge["Pixels"].apply(lambda x: stats.percentileofscore(nb_merge["Pixels"], x)/100)
nb_merge["Price percentile"] = nb_merge["Price_euros"].apply(lambda x: stats.percentileofscore(nb_merge["Price_euros"], x)/100)

In [3]:
nb_merge["performance"] = nb_merge.apply(lambda row:
        0.5 * row["Cpu percentile"] + 0.4 * row["Gpu percentile"] + 0.1 * row["Ram percentile"], axis=1)
nb_merge["portability"] = nb_merge.apply(lambda row:
         0.7 * (1-row["Weight percentile"]) + 0.3 * (1-row["Size percentile"]), axis=1)
nb_merge["battery_qual"] = nb_merge.apply(lambda row:
        0.3 * (1-row["Gpu percentile"]) + 0.3 * (1-row["Cpu percentile"]) + 0.2 * row["Size percentile"] + 0.1 * row["Price percentile"], axis=1)
nb_merge["screen_qual"] = nb_merge.apply(lambda row:
        0.8 * row["Pixel percentile"] + 0.2 * row["Size percentile"], axis=1)

In [4]:
nb_merge

,Unnamed: 0,Unnamed: 0.1,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,...,Weight percentile,Ram percentile,Size percentile,Pixels,Pixel percentile,Price percentile,performance,portability,battery_qual,screen_qual
0,0,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Core i5 7200U,8.0,256GB SSD,...,0.349289,0.514218,0.597630,1920,0.585782,0.120379,0.539289,0.576209,0.394597,0.588152
1,1,133,Dell,Inspiron 5567,Notebook,15.6,1366x768,Core i5 7200U,8.0,1TB HDD,...,0.740758,0.514218,0.597630,1366,0.121327,0.170616,0.539289,0.302180,0.399621,0.216588
2,2,804,Lenovo,ThinkPad X1,2 in 1 Convertible,14.0,IPS Panel Touchscreen 2560x1440,Core i5 7200U,8.0,256GB SSD,...,0.178673,0.514218,0.256872,2560,0.968246,0.928910,0.539289,0.797867,0.407299,0.825972
3,3,375,HP,Elitebook 1040,Ultrabook,14.0,IPS Panel Full HD 1920x1080,Core i7 7500U,8.0,256GB SSD,...,0.143602,0.514218,0.256872,1920,0.585782,0.804739,0.622227,0.822417,0.345118,0.520000
4,4,445,Lenovo,ThinkPad T470s,Ultrabook,14.0,2560x1440,Core i7 7500U,24.0,512GB SSD,...,0.129858,0.982938,0.256872,2560,0.968246,0.934597,0.669100,0.832038,0.358104,0.825972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,1050,365,Dell,Latitude 7480,Notebook,14.0,Full HD 1920x1080,Core i5 7300U,8.0,256GB SSD,...,0.143602,0.514218,0.256872,1920,0.585782,0.694313,0.369668,0.822417,0.503839,0.520000
1051,1051,1100,HP,EliteBook 840,Notebook,14.0,Full HD 1920x1080,Core i5 6200U,4.0,500GB HDD,...,0.229858,0.111374,0.256872,1920,0.585782,0.470142,0.098957,0.762038,0.642796,0.520000
1052,1052,1267,Dell,XPS 13,2 in 1 Convertible,13.3,Quad HD+ / Touchscreen 3200x1800,Core i5 7Y54,8.0,256GB SSD,...,0.070616,0.514218,0.102370,3200,0.989573,0.827488,0.228768,0.919858,0.570711,0.812133
1053,1053,839,Razer,Blade Pro,Gaming,17.3,4K Ultra HD / Touchscreen 3840x2160,Core i7 7820HK,32.0,512GB SSD,...,0.955450,0.991943,0.928910,4,0.018957,0.999052,0.764976,0.052512,0.461137,0.200948


In [5]:
nb_merge.to_csv("notebook_dataset.csv")